In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse #renamed to urlib.parse in python 3

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from pathlib import Path

In [ ]:
url = "https://www.airbnb.com.sg/s/Bangkok--Thailand/homes?refinement_paths%5B%5D=%2Fhomes&checkin=2024-10-04&checkout=2024-10-10&adults=3&tab_id=home_tab&query=Bangkok%2C%20Thailand&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-08-01&monthly_length=3&monthly_end_date=2024-11-01&price_filter_input_type=0&price_filter_num_nights=6&channel=EXPLORE&date_picker_type=calendar&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&source=structured_search_input_header&search_type=user_map_move&search_mode=regular_search&ne_lat=13.810393924485789&ne_lng=100.5597668742775&sw_lat=13.71428581582692&sw_lng=100.48846329668004&zoom=12.786708747368644&zoom_level=12.786708747368644&search_by_map=true"
# url = "https://www.airbnb.com.sg/s/Indonesia/homes?refinement_paths%5B%5D=%2Fhomes&checkin=2024-07-25&checkout=2024-07-27&adults=2&children=0"
parsed_url = urlparse(url)
root_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
driver = webdriver.Chrome()
driver.get(url)
try:
    element = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, "//div[@itemprop='itemListElement']"))
)

finally:
    html_content = driver.page_source
    driver.quit()
# time.sleep(5)
# html_content = driver.page_source
# driver.quit()

soup=BeautifulSoup(html_content,'html.parser')

listings = soup.find_all("div",{"itemprop":"itemListElement"})
for listing in listings:
    title = listing.find('div',{'data-testid':'listing-card-title'}).text.strip()
    rating = listing.find('div',{'class':'t1a9j9y7'}).text.strip().split()[0]
    price = listing.find('span',{"class":"_11jcbg2"}).text.strip().split()[0]
    price_tax = listing.find('div',{'class':'_i5duul'}).find('div',{"class":"_10d7v0r"}).text.strip().split()[0]
    link = listing.find('a').get('href')
    # Reconstruct the absolute link
    link = root_url+link
    print(f"Title: {title}\nRating: {rating}/5\nPrice: {price}\nTotal Price: {price_tax}\nLink: {link} \n\n")

In [9]:
url = "https://www.airbnb.com.sg/s/Bangkok--Thailand/homes?refinement_paths%5B%5D=%2Fhomes&checkin=2024-10-04&checkout=2024-10-10&adults=3&tab_id=home_tab&query=Bangkok%2C%20Thailand&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-08-01&monthly_length=3&monthly_end_date=2024-11-01&price_filter_input_type=0&price_filter_num_nights=6&channel=EXPLORE&date_picker_type=calendar&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&source=structured_search_input_header&search_type=user_map_move&search_mode=regular_search&ne_lat=13.810393924485789&ne_lng=100.5597668742775&sw_lat=13.71428581582692&sw_lng=100.48846329668004&zoom=12.786708747368644&zoom_level=12.786708747368644&search_by_map=true"
# url = "https://www.airbnb.com.sg/s/Indonesia/homes?refinement_paths%5B%5D=%2Fhomes&checkin=2024-07-25&checkout=2024-07-27&adults=2&children=0"
parsed_url = urlparse(url)
root_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
driver = webdriver.Chrome()
driver.get(url)
airbnb_data = pd.DataFrame(columns=["Title", "Rating", "Price/night (SGD)", "Total Price (SGD)", "Link"])
try:
    while True:
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@itemprop='itemListElement']"))
        )
        pagination = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "p1j2gy66"))
        )
        html_content = driver.page_source
        soup=BeautifulSoup(html_content,'html.parser')

        listings = soup.find_all("div",{"itemprop":"itemListElement"})
        for listing in listings:
            try:
                title = listing.find('div',{'data-testid':'listing-card-title'}).text.strip()
                rating = listing.find('div',{'class':'t1a9j9y7'}).text.strip().split()[0]
                price = listing.find('span',{"class":"_11jcbg2"}).text.strip().split()[0]
                price_tax = listing.find('div',{'class':'_i5duul'}).find('div',{"class":"_10d7v0r"}).text.strip().split()[0]
                link = listing.find('a').get('href')
                # Reconstruct the absolute link
                link = root_url+link
                current_data = pd.DataFrame({
                    "Title":title,
                    "Rating":rating,
                    "Price/night (SGD)":price,
                    "Total Price (SGD)":price_tax,
                    "Link":[link]
                })
                airbnb_data = pd.concat([airbnb_data, current_data], axis = 0)
                # print(f"Title: {title}\nRating: {rating}/5\nPrice: {price}\nTotal Price: {price_tax}\nLink: {link} \n\n")
            
            # skip the current listing if contain missing info/error
            except:
                continue
        button = soup.find('a',{'aria-label':'Next'})
        # check for last page
        if button:
            button = root_url+button.get('href')
            driver.get(button)
        else:
            break
   
finally:
    driver.quit()
    file_path = Path('./dataset/airbnb.csv')
    file_path.parent.mkdir(parents = True, exist_ok = True)
    airbnb_data.to_csv(file_path, index = False)

In [18]:
data = pd.read_csv("./dataset/airbnb.csv")
data["Price/night (SGD)"] = data["Price/night (SGD)"].str.replace("$", "").astype('int')
print(data.sort_values(by="Price/night (SGD)").head())

                                 Title Rating  Price/night (SGD)  \
214  Place to stay in Khet Phra Nakhon   4.45                 22   
98                     Flat in Bangkok   4.73                 31   
192  Flat in Khet Pom Prap Sattru Phai   4.64                 31   
244            Hostel in Khet Bang Rak   4.52                 32   
110  Flat in Khet Pom Prap Sattru Phai   4.76                 33   

    Total Price (SGD)                                               Link  
214              $142  https://www.airbnb.com.sg/rooms/93675883819483...  
98               $199  https://www.airbnb.com.sg/rooms/21942959?adult...  
192              $199  https://www.airbnb.com.sg/rooms/11722665?adult...  
244              $191  https://www.airbnb.com.sg/rooms/31764152?adult...  
110              $212  https://www.airbnb.com.sg/rooms/21892849?adult...  
